# cancer data的数据准备

In [2]:
from tqdm import tqdm
import pickle as pk
# 保证文献有作者和V
A_max = 753072
P_max = 412500
V_max = 545982

In [43]:
f = open("../data/academic/paper_author.csv", "r")
paper_author ={}
author_paper ={}
for line in f:
    line = line.strip().split(',')
    if line[0] == 'PMID':
        continue
    elif line[0] in paper_author:
        paper_author[line[0]].append(line[3])
    else:
        paper_author[line[0]] = [line[3]]
    if line[3] in author_paper:
        author_paper[line[3]].append(line[0])
    else:
        author_paper[line[3]] = [line[0]]
f.close()
# 0[pmid],3[aid]

In [44]:
f = open("../data/academic/paper_bioentity.csv", "r")
paper_bioentity = {}
bioentity_paper = {}
for line in f:
    line = line.strip().split(',')
    if line[0] == 'PMID':
        continue
    elif line[0] in paper_bioentity:
        paper_bioentity[line[0]].append(line[1])
    else:
        paper_bioentity[line[0]] = [line[1]]
    if line[1] in bioentity_paper:
        bioentity_paper[line[1]].append(line[0])
    else:
        bioentity_paper[line[1]] = [line[0]]
f.close()
# 0[pmid],1[mention]

In [45]:
# 只保留既有author关系，也有entity关系的文献
PID_set = paper_author.keys() & paper_bioentity.keys()
len(PID_set)

412500

In [47]:
# 重复上边的代码，重新弄一下；限定到PID
f = open("../data/academic/paper_author.csv", "r")
paper_author ={}
author_paper ={}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] not in PID_set:
        continue
    if line[0] in paper_author:
        paper_author[line[0]].append(line[3])
    else:
        paper_author[line[0]] = [line[3]]
    if line[3] in author_paper:
        author_paper[line[3]].append(line[0])
    else:
        author_paper[line[3]] = [line[0]]
f.close()
# 0[pmid],3[aid]

1761904it [00:10, 172895.61it/s]


In [48]:
f = open("../data/academic/paper_bioentity.csv", "r")
paper_bioentity = {}
bioentity_paper = {}
for line in f:
    line = line.strip().split(',')
    if line[0] not in PID_set:
        continue
    if line[0] in paper_bioentity:
        paper_bioentity[line[0]].append(line[1])
    else:
        paper_bioentity[line[0]] = [line[1]]
    if line[1] in bioentity_paper:
        bioentity_paper[line[1]].append(line[0])
    else:
        bioentity_paper[line[1]] = [line[0]]
f.close()
# 0[pmid],1[mention]

In [49]:
len(paper_author.keys() & paper_bioentity.keys())

412500

In [50]:
len(author_paper),len(bioentity_paper)

(753072, 545982)

In [52]:
# 将id变为顺序数字index（Pcount）
PID_trans_Pcount = {}
Pcount_trans_PID = {}

#paper从0开始
count = 0
for PID in PID_set:
    PID_trans_Pcount[PID] = count
    Pcount_trans_PID[count] = PID
    count += 1
pk.dump(PID_trans_Pcount,open('PID_trans_Pcount.pkl','wb'))
pk.dump(Pcount_trans_PID,open('Pcount_trans_PID.pkl','wb'))
# author 从0开始


count = 0
AID_trans_Acount = {}
Acount_trans_AID = {}

for AID in author_paper.keys():
    AID_trans_Acount[AID] = count
    Acount_trans_AID[count] = AID
    count += 1
pk.dump(AID_trans_Acount,open('AID_trans_Acount.pkl','wb'))
pk.dump(Acount_trans_AID,open('Acount_trans_AID.pkl','wb'))
# bioentity 从0开始
count = 0
BID_trans_Bcount = {}
Bcount_trans_BID = {}
for BID in bioentity_paper.keys():
    BID_trans_Bcount[BID] = count
    Bcount_trans_BID[count] = BID
    count += 1
pk.dump(BID_trans_Bcount,open('BID_trans_Bcount.pkl','wb'))
pk.dump(Bcount_trans_BID,open('Bcount_trans_BID.pkl','wb'))

In [56]:
Pcount_trans_PID = pk.load(open('Pcount_trans_PID.pkl','rb'))

In [58]:
paper_c2author_c = {}
author_c2paper_c = {}
paper_c2bio_c = {}
bio_c2paper_c = {}
for k,v in paper_author.items():
    # k,PId; V AID；下面转化为数字符号；实际上只转换bioentity应该就可以，在此全部转换，以防万一
    paper_c2author_c[PID_trans_Pcount[k]] = list(map(lambda x: AID_trans_Acount[x],v))
# 以下基本相同
for k,v in author_paper.items():
    author_c2paper_c[AID_trans_Acount[k]] = list(map(lambda x: PID_trans_Pcount[x],v))

for k,v in paper_bioentity.items():
    paper_c2bio_c[PID_trans_Pcount[k]] = list(map(lambda x: BID_trans_Bcount[x],v))

for k,v in bioentity_paper.items():
    bio_c2paper_c[BID_trans_Bcount[k]] = list(map(lambda x: PID_trans_Pcount[x],v))

In [ ]:
# p_a_list_train = [[] for k in range(P_max)]
# a_p_list_train = [[] for k in range(A_max)]
# v_p_list_train = [[] for k in range(V_max)]

In [59]:
p_a_list_train_f = open("../data/academic/p_a_list_train.txt", "w")
import operator
sorted_paper_c2author_c = sorted(paper_c2author_c.items(),key=operator.itemgetter(0))
for k, v in  dict(sorted_paper_c2author_c).items():
    p_a_list_train_f.write(str(k) + ":")
    for tt in range(len(v)-1):
        p_a_list_train_f.write(str(v[tt]) + ",")
    p_a_list_train_f.write(str(v[-1]))
    p_a_list_train_f.write("\n")
p_a_list_train_f.close()

In [60]:
a_p_list_train_f = open("../data/academic/a_p_list_train.txt", "w")
sorted_author_c2paper_c = sorted(author_c2paper_c.items(),key=operator.itemgetter(0))
for k, v in  dict(sorted_author_c2paper_c).items():
    a_p_list_train_f.write(str(k) + ":")
    for tt in range(len(v)-1):
        a_p_list_train_f.write(str(v[tt]) + ",")
    a_p_list_train_f.write(str(v[-1]))
    a_p_list_train_f.write("\n")
a_p_list_train_f.close()

In [61]:
v_p_list_train_f = open("../data/academic/v_p_list_train.txt", "w")
sorted_bio_c2paper_c = sorted(bio_c2paper_c.items(),key=operator.itemgetter(0))
for k, v in  dict(sorted_bio_c2paper_c).items():
    v_p_list_train_f.write(str(k) + ":")
    for tt in range(len(v)-1):
        v_p_list_train_f.write(str(v[tt]) + ",")
    v_p_list_train_f.write(str(v[-1]))
    v_p_list_train_f.write("\n")
v_p_list_train_f.close()

In [62]:
p_v_f = open("../data/academic/p_v.txt", "w")
sorted_paper_c2bio_c = sorted(paper_c2bio_c.items(),key=operator.itemgetter(0))
for k, v in  dict(sorted_paper_c2bio_c).items():
    p_v_f.write(str(k) + ":")
    for tt in range(len(v)-1):
        p_v_f.write(str(v[tt]) + ",")
    p_v_f.write(str(v[-1]))
    p_v_f.write("\n")
p_v_f.close()

In [ ]:
# f = open("../data/academic/node_net_embedding.txt", "r")


# v_dict = {}
# for line in f:
#     line = line.strip().split(' ')
#     if len(line) != 129 or line[0][0] !='v':
#         continue
#     v_dict[int(line[0][1:])] = line[1:]
# f.close()
# f = open("../data/academic/node_net_embedding.txt", "r")

# a_dict = {}
# for line in f:
#     line = line.strip().split(' ')
#     if len(line) != 129 or line[0][0] !='a':
#         continue
#     a_dict[int(line[0][1:])] = line[1:]
# f.close()
# f = open("../data/academic/node_net_embedding.txt", "r")

# # 0[pmid],3[aid]
# p_dict = {}
# for line in f:
#     line = line.strip().split(' ')
#     if len(line) != 129 or line[0][0] !='p':
#         continue
#     p_dict[int(line[0][1:])] = line[1:]
# f.close()

In [ ]:
# f = open("../data/academic/deepwalk_author_embeddings.txt", "w")
# for k,v in a_dict.items():
#     f.write('a'+str(k))
#     for i in v:
#         f.write(' '+i)
#     f.write('\n')

In [3]:
print('hello world')
Pcount_trans_PID = pk.load(open('Pcount_trans_PID.pkl','rb'))
# PID_trans_Pcount = pk.load(open('PID_trans_Pcount.pkl','rb'))
# AID_trans_Acount = pk.load(open('AID_trans_Acount.pkl','rb'))
Acount_trans_AID = pk.load(open('Acount_trans_AID.pkl','rb'))
# BID_trans_Bcount = pk.load(open('BID_trans_Bcount.pkl','rb'))
Bcount_trans_BID = pk.load(open('Bcount_trans_BID.pkl','rb'))

hello world


In [10]:
f = open("../data/academic/0_node_embedding.txt", "r")
fa = open("../data/academic/author_processed_0_node_embedding.txt", "w")
fp = open("../data/academic/paper_processed_0_node_embedding.txt", "w")
fb = open("../data/academic/entity_processed_0_node_embedding.txt", "w")
for line in tqdm(f):
    line = line.strip().split(' ')
    if line[0][0] == 'a':
        fa.write(Acount_trans_AID[int(line[0][1:])])
        for i in line[1:]:
            fa.write(' '+i)
        fa.write('\n')
        continue
    if line[0][0] == 'v':
        fb.write(Bcount_trans_BID[int(line[0][1:])])
        for i in line[1:]:
            fb.write(' '+i)
        fb.write('\n')
        continue
    if line[0][0] == 'p':
        fp.write(Pcount_trans_PID[int(line[0][1:])])
        for i in line[1:]:
            fp.write(' '+i)
        fp.write('\n')
f.close()
fa.close()
fp.close()
fb.close()

1711551it [00:37, 45108.58it/s]
